In [38]:
# importing libraries
import unicodedata
import re
from IPython.display import JSON
import datetime
from datetime import date
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from api_keys import *

In [2]:
url = "https://worldpopulationreview.com/world-cities"
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, "html.parser")
# parent = soup.find(class_ = 'jsx-3940296686') This is old - it changed ! 
parent = soup.find(class_ = 'jsx-3979628367') # always wacth out if this class_ = " " is the same , it may vhange from time to time.

In [3]:
# parent.contents[1]

In [4]:
children = parent.contents[1]

In [5]:
children = parent.contents[1]
dictionary = {"id" : [], "City": [],"Country":[],"Population_2021":[], "Population_2020" :[], "difference_of" :[]}

In [7]:
# child.contents[0] = Rank # 1,2,3...
# child.contents[1] = City
# child.contents[2] = Country
# child.contents[3] =  Population 2021
# child.contents[4] =  Poppulation 2020
# child.contents[5] = Change
# child.contents[6+] = out of RANGE ! ! !
for i, child in enumerate(children):
        dictionary['id'].append(child.contents[0].get_text(strip=True))
        dictionary['City'].append(child.contents[1].get_text(strip=True))
        dictionary['Country'].append(child.contents[2].get_text(strip=True))
        dictionary['Population_2021'].append(child.contents[3].get_text(strip=True))
        dictionary['Population_2020'].append(child.contents[4].get_text(strip=True))
        dictionary['difference_of'].append(child.contents[5].get_text(strip=True))

In [8]:
table = pd.DataFrame.from_dict(dictionary)

In [9]:
# the Population was "object" which is why we transform it to numeric
table.Population_2020 = pd.to_numeric(table.Population_2020.str.replace(',',''))
table.Population_2021 = pd.to_numeric(table.Population_2021.str.replace(',',''))
table.id = pd.to_numeric(table.id)
# table.Population_2021 = pd.to_numeric(table.Population_2021.str.split('|').str[0].str.replace(',',''))

In [10]:
# We create a new column for the country codes.
table['Country_code'] = "1"

In [11]:
table.head(2)

,id,City,Country,Population_2021,Population_2020,difference_of,Country_code
0,1,Tokyo,Japan,37339804,37393128,-0.14%,1
1,2,Delhi,India,31181376,30290936,2.94%,1


In [154]:
# table.info()

### Getting country codes

In [12]:
url = "https://www.iban.com/country-codes"
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, "html.parser")

In [13]:
def get_country_codes(soup):
    dictionary_country_code = {"Country":[],"Country_code":[]}
    table_html = soup.find('table',id="myTable")
    for row in table_html.find_all('tr'):
        for j, column in enumerate(row.find_all('td')):
            if (j==0):
                dictionary_country_code['Country'].append(column.get_text())
            if(j==1):
                dictionary_country_code['Country_code'].append(column.get_text())
    table_country_codes = pd.DataFrame.from_dict(dictionary_country_code)
    return table_country_codes

In [14]:
table_country_codes = get_country_codes(soup)

In [13]:
# table_country_codes

In [15]:
big_table = table.merge(table_country_codes,how='left')

### Example with split string etc

In [16]:
big_table.head(2)

,id,City,Country,Population_2021,Population_2020,difference_of,Country_code
0,1,Tokyo,Japan,37339804,37393128,-0.14%,1
1,2,Delhi,India,31181376,30290936,2.94%,1


### Coutnries NOt existed with the same name in tables :D 

In [17]:
unordered_list_countries = ["Iran","United States","United Kingdom","Russia","Trinidad And Tobago","North Macedonia","Guinea Bissau","Macau","Laos","Palestine","Central African Republic","Republic of the Congo","United Arab Emirates","DR Congo","Venezuela","Philippines","Sudan","South Korea","North Korea","Vietnam","Tanzania","Ivory Coast","Taiwan","Dominican Republic","Syria","Bolivia","Niger","Czech Republic","Netherlands"]
unordered_list_countries_codes = ["IR","US","GB","RU","TT","MK","GW","MO","LA","PS","CF","CG","AE","CD","VE","PH","SD","KR","KP","VN","TZ","CI","TW","DO","SY","BO","NE","CZ","NL"]

##### Get all countries where the name of country is a match

In [18]:
for i,country in enumerate(table_country_codes.Country):
    for y , city in enumerate(big_table.City):
        if table_country_codes.Country[i] == table.Country[y]:
             big_table.loc[y,'Country_code']=table_country_codes.Country_code[i]

##### Get the rest of countries where the name of country is NOT a match in both tables.

In [19]:
for i,item in enumerate(big_table.Country):
    for y,index in enumerate(unordered_list_countries):
        if item ==index:
                big_table.loc[i,'Country_code'] = unordered_list_countries_codes[y]

In [20]:
big_table['CityCountry'] = big_table[['City','Country_code']].apply(lambda x: ', '.join(x), axis = 1)

In [21]:
big_table["City"].duplicated().sum()

7

In [45]:
# big_table.info()

In [22]:
# big_table[(~big_table.duplicated())]
big_table = big_table.drop_duplicates(subset=['City'], keep='first')

In [23]:
big_table.shape

(1163, 8)

In [27]:
big_table["CityCountry"].duplicated().sum()

0

In [33]:
# Here we check if there is any row left with value of "1" we defined earlier.
# If yes , this would mean that there are one or more countries which they hadn't been given the country code.
big_table.query('Country_code =="1"')

,id,City,Country,Population_2021,Population_2020,difference_of,Country_code,CityCountry


In [51]:
# big_table.to_csv('Cities_countries.csv' ,index=False)

# Cities to sql

In [141]:
cities = big_table.loc[:,["id","City","Country","Country_code","CityCountry"]]

In [142]:
cities.shape

(1163, 5)

In [143]:
cities.head()

,id,City,Country,Country_code,CityCountry
0,1,Tokyo,Japan,JP,"Tokyo, JP"
1,2,Delhi,India,IN,"Delhi, IN"
2,3,Shanghai,China,CN,"Shanghai, CN"
3,4,Sao Paulo,Brazil,BR,"Sao Paulo, BR"
4,5,Mexico City,Mexico,MX,"Mexico City, MX"


In [144]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1163 entries, 0 to 1169
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1163 non-null   int64 
 1   City          1163 non-null   object
 2   Country       1163 non-null   object
 3   Country_code  1163 non-null   object
 4   CityCountry   1163 non-null   object
dtypes: int64(1), object(4)
memory usage: 54.5+ KB


In [160]:
schema="gans"
host="127.0.0.1"
user="root"
password= sql_pass
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
cities.to_sql('cities',con=con,if_exists='append',index=False)

# POPULATION to sql

## Creating a sub - DataFrame from from existing one

In [119]:
big_table.shape

(1163, 8)

In [112]:
population = big_table.query('City == ("Athens","Berlin","Barcelona","Istanbul","Moscow","London","Zurich","Paris","Rome")')

In [113]:
population.to_csv('Population_big_Cities.csv', index=False)

In [134]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 12 to 366
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               9 non-null      int64 
 1   City             9 non-null      object
 2   Country          9 non-null      object
 3   Population_2021  9 non-null      int64 
 4   Population_2020  9 non-null      int64 
 5   difference_of    9 non-null      object
 6   Country_code     9 non-null      object
 7   CityCountry      9 non-null      object
dtypes: int64(3), object(5)
memory usage: 648.0+ bytes


##### And this here is the code to send our tables to the MySQL

In [161]:
schema="gans"
host="127.0.0.1"
user="root"
password= sql_pass
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
# var_name.to_sql('population',con=con,if_exists='append',index=False)
big_table.to_sql('population',con=con,if_exists='append',index=False)


# var_name : the name of variable that contains our dataframe 
# 'population' : the name of the table we WANT to send to the SQL

# about the if_exists : replace , append or fail by default is fail.

# for more info : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

## Weather API

In [149]:
path = "C:/Users/Kosta/OneDrive/Desktop/Data_Science"
from pyowm.owm import OWM
import sys
sys.path.insert(1, path)

In [150]:
pd.set_option('display.expand_frame_repr', False)
owm = OWM(weather_key)

In [151]:
some_cities = cities.query('City == ("Athens","Berlin","Barcelona","Istanbul","Moscow","London","Zurich","Paris","Rome")')

In [152]:
def get_weather_tables(cities):
    owm = OWM(weather_key)
    city_weather_tables = pd.DataFrame()
    for index, row in cities.iterrows():
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={row['CityCountry']}&appid={weather_key}&units=metric") 
        response = requests.get(url)
        json = response.json()
        dictionary = {"Date_Time":[],'Temperature':[],'Feels_like': [],'Humidity':[],'Weather':[],'Clouds':[],'Wind_speed':[],'Rain':[],'Snow':[]}
        for hour in json['list']:
            dictionary['Date_Time'].append(hour['dt_txt'])
            dictionary['Temperature'].append(hour['main']['temp'])
            dictionary['Feels_like'].append(hour['main']['feels_like'])
            dictionary['Humidity'].append(hour['main']['humidity'])
            dictionary['Weather'].append(hour['weather'][0]['description'])
            dictionary['Clouds'].append(hour['clouds']['all'])
            dictionary['Wind_speed'].append(hour['wind']['speed'])
            try:
                dictionary['Rain'].append(hour['rain']['3h'])
            except:
                dictionary['Rain'].append('0')
            try:
                dictionary['Snow'].append(hour['snow']['3h'])
            except:
                dictionary['Snow'].append('0')
        weather_table=pd.DataFrame.from_dict(dictionary)
        weather_table["Date_Time"] = pd.to_datetime(weather_table["Date_Time"])
        weather_table = weather_table.assign(City_id = row['id'], City_country = row['CityCountry'])
        city_weather_tables = city_weather_tables.append(weather_table)
    return city_weather_tables

In [153]:
def create_mysql_table_weather(big_table):
    schema="gans"
    host="127.0.0.1"
    user="root"
    password= sql_pass
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    big_table.to_sql('weather',con=con,if_exists='append',index=False)

In [156]:
create_mysql_table_weather(get_weather_tables(some_cities))

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'City_country' in 'field list'")
[SQL: INSERT INTO weather (`Date_Time`, `Temperature`, `Feels_like`, `Humidity`, `Weather`, `Clouds`, `Wind_speed`, `Rain`, `Snow`, `City_id`, `City_country`) VALUES (%(Date_Time)s, %(Temperature)s, %(Feels_like)s, %(Humidity)s, %(Weather)s, %(Clouds)s, %(Wind_speed)s, %(Rain)s, %(Snow)s, %(City_id)s, %(City_country)s)]
[parameters: ({'Date_Time': datetime.datetime(2022, 1, 22, 18, 0), 'Temperature': 0.73, 'Feels_like': -2.35, 'Humidity': 60, 'Weather': 'scattered clouds', 'Clouds': 40, 'Wind_speed': 2.7, 'Rain': '0', 'Snow': '0', 'City_id': 13, 'City_country': 'Istanbul, TR'}, {'Date_Time': datetime.datetime(2022, 1, 22, 21, 0), 'Temperature': 0.67, 'Feels_like': -3.0, 'Humidity': 59, 'Weather': 'scattered clouds', 'Clouds': 44, 'Wind_speed': 3.37, 'Rain': '0', 'Snow': '0', 'City_id': 13, 'City_country': 'Istanbul, TR'}, {'Date_Time': datetime.datetime(2022, 1, 23, 0, 0), 'Temperat

In [157]:
# example_table = get_weather_tables(some_cities)
# example_table

In [159]:
example_table.head()

,Date_Time,Temperature,Feels_like,Humidity,Weather,Clouds,Wind_speed,Rain,Snow,City_id,City_country
0,2022-01-22 18:00:00,0.73,-2.35,60,scattered clouds,40,2.70,0,0,13,"Istanbul, TR"
1,2022-01-22 21:00:00,0.67,-3.00,59,scattered clouds,44,3.37,0,0,13,"Istanbul, TR"
2,2022-01-23 00:00:00,0.47,-3.54,66,light snow,51,3.77,0,0.22,13,"Istanbul, TR"
3,2022-01-23 03:00:00,0.99,-3.45,63,light snow,96,4.61,0,0.16,13,"Istanbul, TR"
4,2022-01-23 06:00:00,1.30,-3.92,59,light snow,97,6.26,0,0.44,13,"Istanbul, TR"


In [162]:
create_mysql_table_weather(example_table)

In [85]:
example_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 39
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date_Time     360 non-null    datetime64[ns]
 1   Temperature   360 non-null    float64       
 2   Feels_like    360 non-null    float64       
 3   Humidity      360 non-null    int64         
 4   Weather       360 non-null    object        
 5   Clouds        360 non-null    int64         
 6   Wind_speed    360 non-null    float64       
 7   Rain          360 non-null    object        
 8   Snow          360 non-null    object        
 9   City_id       360 non-null    int64         
 10  City_country  360 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 33.8+ KB


In [138]:
big_table.shape

(1163, 8)

In [191]:
cities.head(1)

,City,Country,Country_code,CityCountry
0,Tokyo,Japan,JP,"Tokyo, JP"


In [107]:
exp_table.head()

,City,Country,Country_code,CityCountry
0,Tokyo,Japan,JP,"Tokyo, JP"
1,Delhi,India,IN,"Delhi, IN"
2,Shanghai,China,CN,"Shanghai, CN"
3,Sao Paulo,Brazil,BR,"Sao Paulo, BR"
4,Mexico City,Mexico,MX,"Mexico City, MX"


In [121]:
exp_table.shape

(1163, 4)

In [342]:
weather = weather_forecast

# FLIGHTS

In [86]:
def get_jsons_from_AerodataboxAPI(icao):
    today = datetime.date.today()
    tomorrow = str(today + datetime.timedelta(days=1))    
    times = [["00:00","11:59"],["12:00","23:59"]]
    jsons = []
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        querystring = {"direction":"Arrival","withPrivate":"true"}
        headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': flight_API_key
            }
        jsons.append(requests.request("GET", url, headers=headers, params=querystring).json())
    return jsons

In [ ]:
# NO need to repeat this
# response_one = get_jsons_from_AerodataboxAPI('EDDB')
# JSON(response_one)
# response_two = (get_jsons_from_AerodataboxAPI('EDDB')).json()
# response_two

### Define some Cities - You cannot add every city for every possible flight due to a limited amount of APIS call per month

In [87]:
# 
def get_cities_dataframe():
    cities = pd.DataFrame({'City': ['Istanbul','Moscow','Berlin'], 'Country': ['Turkey','Russian Federation (the)','Germany'], 
    'Country_code': ['TR','RU','DE'],'CityCountry' : ["Istanbul, TR","Moscow, RU","Berlin, DE"],
    'Country_short' : ['TR','RU','DE'], 'icao' : ["LTFM","UUDD","EDDB"]})
    return cities[:3]

In [88]:
def create_dictionary_from_json(jsons):
    dictionary = {"From_location":[],"Arrival_at":[], "Terminal":[],"Airline":[],"Flight_number":[],"Airport_code":[]}
    for resp in jsons:
        for flight in resp['arrivals']:
            try:
                dictionary["From_location"].append(flight['movement']["airport"]['name'])
            except:
                dictionary["From_location"].append("Unknown")
            try:
                dictionary["Arrival_at"].append(flight['movement']['scheduledTimeLocal'][:-6])
            except:
                dictionary["Arrival_at"].append("Unknown")
            try:
                dictionary["Terminal"].append(flight['movement']['terminal'])
            except:
                dictionary["Terminal"].append("Unknown")
            try:
                dictionary["Airline"].append(flight['airline']['name'])
            except:
                dictionary["Airline"].append("Unknown")
            try:
                dictionary["Flight_number"].append(flight['number'])
            except:
                dictionary["Flight_number"].append("Unknown")
            try:
                dictionary["Airport_code"].append(flight['movement']["airport"]["icao"])
            except:
                dictionary["Airport_code"].append("Unknown")
    return dictionary 

In [89]:
def create_dataframe_from_dictionaries(dictionary):
    flights = pd.DataFrame.from_dict(dictionary)
    return flights

In [90]:
def get_all_info(cities):
    big_table = pd.DataFrame()
    for index, city in cities.iterrows():
        jsons = get_jsons_from_AerodataboxAPI(city['icao'])
        dictionary = create_dictionary_from_json(jsons)
        dataframe = create_dataframe_from_dictionaries(dictionary)
        dataframe = dataframe.assign(CityCountry = city['CityCountry'])
        dataframe = dataframe.assign(icao = city['icao'])
        dataframe["Arrival_at"] = pd.to_datetime(dataframe["Arrival_at"])
#         dataframe["Arrival_UTC_time"] = pd.to_datetime(dataframe["Arrival_UTC_time"])
    big_table = big_table.append(dataframe,ignore_index=True)
    return big_table

In [165]:
def create_mysql_table_flights(big_table):
    schema="gans"
    host="127.0.0.1"
    user="root"
    password=sql_pass
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    big_table.to_sql('flights',con=con,if_exists='append',index=False)

In [ ]:
# temp_csv_flights = get_all_info(get_cities_dataframe())

In [166]:
create_mysql_table_flights(temp_csv_flights)

In [ ]:
# create_mysql_table(get_all_info(get_cities_dataframe()))

In [96]:
# temp_csv_flights.to_csv('flights.csv',index=False)

In [344]:
# weather.to_sql('weather',con=con,if_exists='append',index=False)

# AIRPORTS to sql

In [54]:
airports = pd.read_csv('airports.csv')
# airports

In [55]:
airports.loc[airports['municipality'] == 'St. Petersburg', 'municipality'] = 'Saint Petersburg'

In [56]:
airports.loc[airports['municipality'] == 'Kiev', 'municipality'] = 'Kyiv'

In [57]:
airports = airports.dropna(subset=['municipality', 'iso_country'])

In [58]:
airports.loc[airports['municipality'].str.contains('Istanbul'), 'municipality'] = 'Istanbul'

In [59]:
list_of_cities = big_table.CityCountry.to_list()

In [60]:
airports.head(1)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN


In [61]:
airports_only_needed = airports[['type','name','latitude_deg','longitude_deg','iso_country','elevation_ft','municipality','scheduled_service','gps_code']]

In [62]:
airports_only_needed = (
    airports_only_needed
            .query("(type == 'medium_airport' | type == 'large_airport') & scheduled_service == 'yes'")                          
)

In [63]:
big_table['CityCountry'] = big_table[['City','Country_code']].apply(lambda x: ', '.join(x), axis = 1)

In [64]:
airports_only_needed['CityCountry'] = airports_only_needed[['municipality','iso_country']].apply(lambda x: ', '.join(x), axis = 1)

In [65]:
airports_only_needed = airports_only_needed[airports_only_needed.CityCountry.isin(list_of_cities)]

In [66]:
airports_only_needed.type = pd.Categorical(airports_only_needed.type, categories=["large_airport","medium_airport"],ordered=True)

In [67]:
airports_only_needed.sort_values('type', inplace=True)

In [68]:
airports_only_needed = airports_only_needed.drop_duplicates(subset=['CityCountry'])

In [69]:
airports_only_needed.shape

(555, 10)

In [311]:
# airports_only_needed

In [70]:
airports_only_needed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 37373 to 68325
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   type               555 non-null    category
 1   name               555 non-null    object  
 2   latitude_deg       555 non-null    float64 
 3   longitude_deg      555 non-null    float64 
 4   iso_country        555 non-null    object  
 5   elevation_ft       540 non-null    float64 
 6   municipality       555 non-null    object  
 7   scheduled_service  555 non-null    object  
 8   gps_code           555 non-null    object  
 9   CityCountry        555 non-null    object  
dtypes: category(1), float64(3), object(6)
memory usage: 44.0+ KB


In [102]:
airports_only_needed.rename(columns={'type': 'airport_type', 'name': 'airport_name'}, inplace=True)

In [136]:
airports_only_needed

,airport_type,airport_name,latitude_deg,longitude_deg,iso_country,elevation_ft,municipality,scheduled_service,gps_code,CityCountry
37373,large_airport,Marseille Provence Airport,43.439272,5.221424,FR,74.0,Marseille,yes,LFML,"Marseille, FR"
38559,large_airport,Esenboğa International Airport,40.128101,32.995098,TR,3125.0,Ankara,yes,LTAC,"Ankara, TR"
38510,large_airport,Zürich Airport,47.458056,8.548056,CH,1417.0,Zurich,yes,LSZH,"Zurich, CH"
38450,large_airport,Geneva Cointrin International Airport,46.238098,6.108950,CH,1411.0,Geneva,yes,LSGG,"Geneva, CH"
38333,large_airport,Henri Coandă International Airport,44.571111,26.085000,RO,314.0,Bucharest,yes,LROP,"Bucharest, RO"
...,...,...,...,...,...,...,...,...,...,...
44592,medium_airport,Lieutenant General Qasem Soleimani Internation...,31.337400,48.762001,IR,66.0,Ahvaz,yes,OIAW,"Ahvaz, IR"
44328,medium_airport,Tabuk Airport,28.365400,36.618900,SA,2551.0,Tabuk,yes,OETB,"Tabuk, SA"
44236,medium_airport,Ahmad Shah Baba International Airport / Kandah...,31.505800,65.847801,AF,3337.0,Kandahar,yes,OAKN,"Kandahar, AF"
43042,medium_airport,Warri Airport,5.596110,5.817780,NG,242.0,Warri,yes,DNSU,"Warri, NG"


In [164]:
schema="gans"
host="127.0.0.1"
user="root"
password=sql_pass
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
airports_only_needed.to_sql('airports', if_exists='append', con=con, index=False)

In [103]:
airports_only_needed.head()

,airport_type,airport_name,latitude_deg,longitude_deg,iso_country,elevation_ft,municipality,scheduled_service,gps_code,CityCountry
37373,large_airport,Marseille Provence Airport,43.439272,5.221424,FR,74.0,Marseille,yes,LFML,"Marseille, FR"
38559,large_airport,Esenboğa International Airport,40.128101,32.995098,TR,3125.0,Ankara,yes,LTAC,"Ankara, TR"
38510,large_airport,Zürich Airport,47.458056,8.548056,CH,1417.0,Zurich,yes,LSZH,"Zurich, CH"
38450,large_airport,Geneva Cointrin International Airport,46.238098,6.108950,CH,1411.0,Geneva,yes,LSGG,"Geneva, CH"
38333,large_airport,Henri Coandă International Airport,44.571111,26.085000,RO,314.0,Bucharest,yes,LROP,"Bucharest, RO"


In [104]:
airports_only_needed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 37373 to 68325
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   airport_type       555 non-null    category
 1   airport_name       555 non-null    object  
 2   latitude_deg       555 non-null    float64 
 3   longitude_deg      555 non-null    float64 
 4   iso_country        555 non-null    object  
 5   elevation_ft       540 non-null    float64 
 6   municipality       555 non-null    object  
 7   scheduled_service  555 non-null    object  
 8   gps_code           555 non-null    object  
 9   CityCountry        555 non-null    object  
dtypes: category(1), float64(3), object(6)
memory usage: 44.0+ KB


In [73]:
# cities